In [1]:
#flops计算法官法
# 常规卷积  input:（Di,Di,M),output:(Do,Do,N),kernel:(K,K,M),FLOPS =  Do× Do× M × N × K × K

# 该方法卷积 FLOPS = (Do× Do× M × N × K × K)/P + (Do× Do× N) ×（M −M/P）

In [ ]:
# Install PyTorch
!pip install torch==1.7.0
# Install MMCV
!pip install mmcv-full==1.3.18 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
#install mmcv  mmcv的版本要修改
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification
!pip install -e . 

In [ ]:
from torch import nn
from mmcv.cnn.bricks.registry import CONV_LAYERS
import numpy as np
import os
import torch, torchvision
import mmcv
from mmcv import Config

from mmcls.datasets.builder import DATASETS
from mmcls.datasets import BaseDataset
from torchvision.datasets import ImageFolder
from mmcls.apis import train_model,inference_model,show_result_pyplot
from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import set_random_seed
from mmcls.models import BACKBONES
from mmcv.cnn import build_conv_layer, build_norm_layer
from mmcls.models.backbones.resnet import ResLayer, ResNet

In [2]:
#卷积模块
@CONV_LAYERS.register_module()
class HetConv2d(nn.Module):
    
    def __init__(self,input_channel,
                 output_channel,
                 kernel_size,
                 stride=1,
                 padding=0,
                 dilation=1,
                 groups=1,
                 bias=True,
                 p=4,
                 **kwargs):
        super(HetConv2d,self).__init__(**kwargs)
        assert output_channel%p == 0 and input_channel%p ==0
        
        self.conv3_3 = nn.Conv2d(input_channel,output_channel,kernel_size,padding =1,groups=p)
        self.conv1_1 = nn.Conv2d(input_channel,output_channel,1,stride=1,padding =0)
        self.conv1_1_ = nn.Conv2d(input_channel,output_channel,1,stride=1,groups = p,padding =0)

    def forward(self,x):
        #1*1可能让结果变大，要用padding=0
        return self.conv3_3(x)+self.conv1_1(x)-self.conv1_1_(x)
    

#真正的实现

class HetConv(nn.Module):
    '''
    将卷积操作拆分成一层一层的执行，再相加，用两个for循环完整卷积操作，就能够实现对于其中通道卷积的修改
    '''
    def __init__(self,in_channels,
                    out_channels,
                    p):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels= out_channels
        self.p = p
        self.weights= nn.ModuleList()
        for i in range(self.out_channels):
            self.weights.append(self.make_one_wegiht(i,p))
    
    def make_one_wegiht(self,n,p):
        weight = nn.ModuleList()
        for i in range(self.in_channels):
            if ((i-n)%p==0):
                weight.append(nn.Conv2d(1,1,3,1,1))
            else:
                weight.append(nn.Conv2d(1,1,1,1,0))
        return weight
    def forward(self,x):
        out = []
        for i in range(self.out_channels):
            out_ = self.weights[i][0](x[:,0:1,:,:])
            for j in range(1,self.in_channels):
                out_ +=self.weights[i][j](x[:,j:j+1,:,:])
            out.append(out_)
        return torch.cat(out,1)

NameError: name 'CONV_LAYERS' is not defined

In [3]:
#使用上面的卷积做一个基础模块，这个是3*3+3*3，名字不能改
#有坑：expansion表示中间是扩大还是缩小的比率，BasicBlock=1 ，Bottleneck =4,如果模型继承了resnet就要确定这个值
class BasicBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 p=4,
                 expansion=1,
                 stride=1,
                 dilation=1,
                 downsample=None,
                 style='pytorch',
                 with_cp=False,
                 conv_cfg=dict(type='HetConv2d'),
                 norm_cfg=dict(type='BN')):
        super(BasicBlock,self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.expansion = expansion
        assert self.expansion == 1
        assert out_channels % expansion == 0
        self.mid_channels = out_channels // expansion
        assert in_channels % p ==0 and out_channels % p ==0
        self.mid_channels = out_channels
        self.stride = stride
        self.dilation = dilation
        self.style = style
        self.with_cp = with_cp
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg

        self.norm1_name, norm1 = build_norm_layer(
            norm_cfg, self.mid_channels, postfix=1)
        self.norm2_name, norm2 = build_norm_layer(
            norm_cfg, out_channels, postfix=2)

        self.conv1 = build_conv_layer(
            conv_cfg,
            in_channels,
            self.mid_channels,
            3,
            stride=stride,
            padding=dilation,
            dilation=dilation,
            bias=False)
        self.add_module(self.norm1_name, norm1)
        self.conv2 = build_conv_layer(
            conv_cfg,
            self.mid_channels,
            out_channels,
            3,
            padding=1,
            bias=False)
        self.add_module(self.norm2_name, norm2)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    @property
    def norm1(self):
        return getattr(self, self.norm1_name)

    @property
    def norm2(self):
        return getattr(self, self.norm2_name)

    def forward(self, x):

        def _inner_forward(x):
            identity = x

            out = self.conv1(x)
            out = self.norm1(out)
            out = self.relu(out)

            out = self.conv2(out)
            out = self.norm2(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            out += identity

            return out

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(_inner_forward, x)
        else:
            out = _inner_forward(x)

        out = self.relu(out)

        return out


NameError: name 'nn' is not defined

In [ ]:
@BACKBONES.register_module()
class HetResNet(ResNet):
    arch_settings = {
        50: (BasicBlock, (3, 4, 6, 3)),
        101: (BasicBlock, (3, 4, 23, 3)),
        152: (BasicBlock, (3, 8, 36, 3))}

    def __init__(self, depth,**kwargs):
        super(HetResNet, self).__init__(depth,**kwargs)

In [4]:
#数据路径
training_data_path='/kaggle/input/10-monkey-species/training/training/'
valid_data_path='/kaggle/input/10-monkey-species/validation/validation/'
#将数据集写入文本中，方便解析
label=os.listdir(training_data_path)
with open('train.txt','w') as f:
    for l in range(len(label)):
        for filename in os.listdir(os.path.join(training_data_path,label[l])):
            line=f'{label[l]}/{filename} {l}\n'
            f.write(line)
with open('valid.txt','w') as f:
    for l in range(len(label)):
        for filename in os.listdir(os.path.join(valid_data_path,label[l])):
            line=f'{label[l]}/{filename} {l}\n'
            f.write(line)#重写数据类，可以将类名加入，也可以不加
#构造数据集，使用BaseDataset作为基类，子类只要实现load_annotations生成[{'img':path,'label':1,'other':data}]格式的数据即可,但是注意字典的
#key以匹配pipeline中的使用
@DATASETS.register_module()
class MyDataset(BaseDataset):
    def load_annotations(self):
        assert isinstance(self.ann_file, str)
        data_infos = []
        with open(self.ann_file) as f:
            samples = [x.strip().split(' ') for x in f.readlines()]
            for filename, gt_label in samples:
                #里面的key是固定要，要匹配LOADPIPELINE中的key
                info = {'img_prefix': self.data_prefix}
                info['img_info'] = {'filename': filename}
                info['gt_label'] = np.array(gt_label, dtype=np.int64)
                data_infos.append(info)
            return data_infos
#自定义类增益的方法
from mmcls.datasets import PIPELINES

#输入的是字典
@PIPELINES.register_module()
class MyTransform(object):
    def __call__(self, results):
        results['dummy'] = True
        # apply transforms on results['img']
        return results
    

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),#读取完成后就变成了np矩阵了
    dict(type='RandomResizedCrop', size=224),#都是用Mmcv中对矩阵的操作实现的
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(type='ToPIL'),
    dict(type='MyTransform'),#如果要随意增益，是针对PIL图，所以前面后面是固定格式
    dict(type='ToNumpy'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),# to_tensor(img.transpose(2, 0, 1))变换通道后变为张量，这个没有[0，1]的归一化
    dict(type='ToTensor', keys=['gt_label']),#直接变为张量
    dict(type='Collect', keys=['img', 'gt_label'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=256),
    dict(type='CenterCrop', crop_size=224),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

NameError: name 'os' is not defined

In [5]:
#读取所有的参数,参数文件就在当下文件夹中
cfg = Config.fromfile('./configs/resnet/resnet50_8xb16_cifar10.py')
#print(f'Config:\n{cfg.pretty_text}')  #输出所有的参数

NameError: name 'Config' is not defined

In [6]:
#数据集参数指定,对其进行修改
cfg.dataset_type = 'MyDataset' #字符串就是数据类


cfg.data.train.type = 'MyDataset'#数据类名，这个是构建数据集的参数字典
cfg.data.train.ann_file = '/kaggle/working/mmclassification/train.txt'#文本文件
cfg.data.train.data_prefix = training_data_path#路径前缀

cfg.data.train.pipeline=[dict(type='LoadImageFromFile'),
            dict(type='RandomResizedCrop', size=224, backend='pillow'),
            dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
            dict(type='Normalize',
                mean=[110.508858/255.0, 109.552668/255.0,84.623747/255.0],
                std=[67.212821/255.0,66.229520/255.0, 66.544232/255.0],
                to_rgb=True),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='ToTensor', keys=['gt_label']),
            dict(type='Collect', keys=['img', 'gt_label'])]


cfg.data.val.type = 'MyDataset'
cfg.data.val.ann_file = '/kaggle/working/mmclassification/valid.txt'
cfg.data.val.data_prefix = valid_data_path
cfg.data.val.pipeline=[dict(type='LoadImageFromFile'),
            dict(type='Resize', size=(256, -1)),
            dict(type='CenterCrop', crop_size=224),
            dict(type='Normalize',
                mean=[110.508858/255.0, 109.552668/255.0,84.623747/255.0],
                std=[67.212821/255.0,66.229520/255.0, 66.544232/255.0],
                to_rgb=True),
            dict(type='ToTensor', keys=['gt_label']),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img','gt_label'])]
# modify num classes of the model in box head
cfg.model.head.num_classes = 10#预测类个数
cfg.model.backbone.type ='HetResNet'
cfg.model.head.in_channels = 512
#cfg.model.backbone.expansion=1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
#cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'   #是否是预训练模型

# Set up working dir to save files and logs.
#cfg.work_dir = './tutorial_exps'  #保存路径

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer = dict(type='Adam', lr=0.1, weight_decay=1e-4)  #学习率设置



#学习率调整  参数一定要传对
cfg.lr_config = dict(policy='step',0.1, step=[30,90])
#cfg.lr_config = dict(policy='exp',gamma=0.1)
#cfg.lr_config = dict(policy='fixed')
#cfg.lr_config = dict(policy='poly',power=1.25, min_lr=0.001)
#cfg.lr_config=dict(policy='CosineAnnealing',min_lr=0.0001)
#cfg.lr_config = dict(policy='CosineRestart',periods=[1,3])
 
cfg.lr_config.warmup='linear'
cfg.lr_config.warmup_iters=500
cfg.lr_config.warmup_ratio=0.001


cfg.runner = dict(type='EpochBasedRunner', max_epochs=100) #训练迭代次数

cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.

# We can set the evaluation interval to reduce the evaluation times

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12
cfg.work_dir = './tutorial_exps'

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)#设备指定
cfg.evaluation =  dict(interval=1, metric='accuracy')
cfg.evaluation.interval = 12
cfg.workflow= [('train', 2), ('val', 1)]# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')  #输出所有的参数


NameError: name 'cfg' is not defined

In [7]:
#执行训练过程
from mmcv.cnn import (ConvModule, build_conv_layer, build_norm_layer,
                      constant_init, kaiming_init)
#构造数据集，它的长度len(data_loaders) == len(workflow)
#datasets = [build_dataset(cfg.data.train)]
datasets = [build_dataset(cfg.data.train),build_dataset(cfg.data.val)]
# Build the detector
model = build_classifier(cfg.model) #建立模型
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES#将类名加入到模型中

# Create work_dir

#data_loader可以进行优化，
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))#创建保存文件
train_model(model, 
            datasets, 
            cfg, 
            distributed=False,
            validate=True) #训练validate=True是用来做eval的钩子的

ModuleNotFoundError: No module named 'mmcv'

In [8]:
#小技巧：给类添加方法
from mmcls.models.classifiers import ImageClassifier
def forward_dummy(self,x):
    x = self.extract_feat(x)
    x =self.head.simple_test(x)
    return x
ImageClassifier.forward_dummy = forward_dummy

forward=model.forward
model.forward=model.forward_dummy

#计算每一层的FLOPS
from mmcv.cnn.utils import get_model_complexity_info
#查看函数支持计算的层有哪些
#model.forward = model.forward_dummy
#需要对HEADs里实现 extra_repr,随便输出什么字符串即可
get_model_complexity_info(model,(3,512,512))

ModuleNotFoundError: No module named 'mmcls'